In [1]:
import numpy as np
import qilisdk
import qutip as qt
import matplotlib.pyplot as plt
import sys

sys.path.append('../src')

import hamiltonian
import dynamics
import plots
import estimator
import VQE

from qilisdk.backends import QutipBackend
from qilisdk.digital import Circuit, M, U1, CNOT, U2, U3, CZ, RX, RZ, H
from qilisdk.functionals.sampling import Sampling, SamplingResult
from qilisdk.functionals.time_evolution import TimeEvolution
from scipy.optimize import minimize

from qilisdk.analog import Schedule, X, Z, Y
from qilisdk.core import ket, tensor_prod
from qilisdk.backends import QutipBackend, CudaBackend
from qilisdk.functionals import TimeEvolution

In [2]:
nqubits = 1
min_w = -1
max_w = 1

H_true, true_weights = hamiltonian.create_random_hamiltonian(nqubits, min_w, max_w, backend='qutip')

psi_0 = dynamics.initialize_down_state(nqubits)

print(H_true.H, "\n")

print("Weights:", true_weights)

# Weights: [-0.66275718 -0.90763743  0.14737293  0.6965676  -0.41580799]

Quantum object: dims=[[2], [2]], shape=(2, 2), type='oper', dtype=CSR, isherm=True
Qobj data =
[[-0.27925975 -0.67967082]
 [-0.67967082  0.27925975]] 

Weights: [-0.67967082 -0.27925975]


In [3]:
ti = 0
times = np.linspace(0.1, 2, 20)
times = [10]
nsteps = 10000
nshots = 1000

final_states = []
z_list = []
x_list = []
timestamp_measurements = []

for tf in times:
    sim = dynamics.time_evolution(H_true, psi_0, ti, tf, nsteps)
    observables = dynamics.calculate_expectation_values(sim, H_true)

    z_list.append(float(observables['Sz'][-1][0]))
    x_list.append(float(observables['Sx'][-1][0]))

    target_state = sim.states[-1]
    final_states.append(target_state)

    #sample from final state
    measurements = dynamics.sample_from_state(target_state, nqubits, nshots)
    timestamp_measurements.append(measurements)


## Build ansatz Hamiltonian

In [4]:
H_initial, initial_weights = hamiltonian.create_random_hamiltonian(nqubits, min_w, max_w, backend='qutip')

# minimize with scipy 
res = minimize(lambda p: VQE.nll_cost(p, nqubits, psi_0, ti, tf, nsteps, timestamp_measurements),
               initial_weights, method='COBYLA',
               options={'maxiter': 200})


print("Optimization finished, success:", res.success, ", loss function:", res.fun)
print("best parameters:", res.x)
print(true_weights)

<class 'qutip.core.qobj.Qobj'>
<class 'qutip.core.qobj.Qobj'>
<class 'qutip.core.qobj.Qobj'>
<class 'qutip.core.qobj.Qobj'>
<class 'qutip.core.qobj.Qobj'>
<class 'qutip.core.qobj.Qobj'>
<class 'qutip.core.qobj.Qobj'>
<class 'qutip.core.qobj.Qobj'>
<class 'qutip.core.qobj.Qobj'>
<class 'qutip.core.qobj.Qobj'>
<class 'qutip.core.qobj.Qobj'>
<class 'qutip.core.qobj.Qobj'>
<class 'qutip.core.qobj.Qobj'>
<class 'qutip.core.qobj.Qobj'>
<class 'qutip.core.qobj.Qobj'>
<class 'qutip.core.qobj.Qobj'>
<class 'qutip.core.qobj.Qobj'>
<class 'qutip.core.qobj.Qobj'>
<class 'qutip.core.qobj.Qobj'>
<class 'qutip.core.qobj.Qobj'>
<class 'qutip.core.qobj.Qobj'>
<class 'qutip.core.qobj.Qobj'>
<class 'qutip.core.qobj.Qobj'>
<class 'qutip.core.qobj.Qobj'>
<class 'qutip.core.qobj.Qobj'>
<class 'qutip.core.qobj.Qobj'>
<class 'qutip.core.qobj.Qobj'>
<class 'qutip.core.qobj.Qobj'>
<class 'qutip.core.qobj.Qobj'>
<class 'qutip.core.qobj.Qobj'>
<class 'qutip.core.qobj.Qobj'>
<class 'qutip.core.qobj.Qobj'>
<class '